In [1]:
import torch
# torchversion = torch.__version__

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader.dataloader import DataLoader
from torch_geometric.datasets import ZINC
from models.GraphTansformerNet import GraphTransformerNet
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

## Train

In [8]:
def train(epoch, loss_func):
    model.train()
    for data in train_loader:
    #for data in tqdm(train_loader, desc="Training Loader"):
        data = data.to(device)
        optimizer.zero_grad()

        out = model(x=data.x.float(), edge_index=data.edge_index, edge_attr=data.edge_attr, batch=data.batch)
        loss = loss_func(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        train_mae = mean_absolute_error(data.y.cpu().detach().numpy(), out.squeeze().cpu().detach().numpy())
    return loss, train_mae


@torch.no_grad()
def test(loader):
    model.eval()
    for data in loader:
    #for data in tqdm(loader, desc="Test Loader"):
        data = data.to(device)

        out = model(x=data.x.float(), edge_index=data.edge_index, edge_attr=data.edge_attr, batch=data.batch)
        loss = criterion(out.squeeze(), data.y)
        test_mae = mean_absolute_error(data.y.cpu().detach().numpy(), out.squeeze().cpu().detach().numpy())
    return loss, test_mae

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
epochs=10
batch_size=32
hidden=64
n_layers=10
heads=8

dataset_train = ZINC(root=f'data/ZINC', split="train", subset=True)
dataset_val = ZINC(root=f'data/ZINC', split="val", subset=True)
dataset_test = ZINC(root=f'data/ZINC', split="test", subset=True)
train_loader = DataLoader(dataset_train, batch_size=batch_size)
val_loader = DataLoader(dataset_val, batch_size=batch_size)
test_loader = DataLoader(dataset_test, batch_size=batch_size)


model = GraphTransformerNet(node_dim=dataset_train.num_features,
                            edge_dim=dataset_train.num_edge_features,
                            hidden_dim=hidden,
                            num_layers=n_layers,
                            num_heads=heads,
                            dropout=0.1)

print(f"Train graphs: {len(dataset_train)}")
print(f"Val graphs: {len(dataset_val)}")
print(f"Test graphs: {len(dataset_test)}\n")
print(model)
total_params = count_parameters(model)
print("Total de parámetros en el modelo:", total_params)

Train graphs: 10000
Val graphs: 1000
Test graphs: 1000

GraphTransformerNet(
  (node_emb): Linear(in_features=1, out_features=64, bias=False)
  (edge_emb): Linear(in_features=1, out_features=64, bias=False)
  (layers): ModuleList(
    (0-9): 10 x GraphTransformerLayer()
  )
  (global_pool): MultiAggregation([
    SumAggregation(),
  ], mode=cat)
  (mlp_readout): MLPReadout(
    (FC_layers): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=16, bias=True)
      (2): Linear(in_features=16, out_features=1, bias=True)
    )
  )
)
Total de parámetros en el modelo: 581953


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                              min_lr=0.00001)

In [11]:
model.to(device)

for epoch in tqdm(range(epochs), desc="Training Graph Transformer"):
    train_loss, train_mae = train(epoch, criterion)
    val_loss, val_mae = test(val_loader)
    scheduler.step(val_loss)
    print(f'Epoch: {epoch+1:03d}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f} | Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f}')

_, test_mae = test(test_loader)
print(f'Test MAE: {test_mae:.4f}')

Training Graph Transformer:   0%|          | 0/10 [00:00<?, ?it/s]

1


Training Graph Transformer:   0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1612 and 1x64)